# Data validation with ORMs

LaminDB implements data validation at the ORM level by fully integrating the SQLModel ORM with pydantic type checking.

Let's take a look at data validation behavior in LaminDB.

In [ ]:
import lamindb.schema as lns
import pytest
from pydantic import ValidationError

## Missing Field Error

Let's create a `User` instance without the required `email` and `handle` fields.

In [ ]:
with pytest.raises(ValidationError) as e:
    exception = e
    user_missing = lns.User(id="123")
print(exception.exconly())

## Simple Type Error

Let's create a `Pipeline` instance with the wrong type for the optional field `name`.

In [ ]:
from datetime import datetime

with pytest.raises(ValidationError) as e:
    exception = e
    invalid_pipeline = lns.Pipeline(name=datetime.now())
print(exception.exconly())

## Simple Categorical Error

Let's pass an invalid categorical to the `type` field in `Usage`, which only accepts the values 'ingest', 'insert', 'select', 'update', 'delete', 'load', and 'link'.

In [ ]:
??lns.Usage

In [ ]:
with pytest.raises(ValidationError) as e:
    exception = e
    invalid_usage = lns.Usage(type="invalid categorical", dobject_id="123")
print(exception.exconly())

## Special Cases in Data Validation

Data validation with the LaminDB ORM mirrors the standard Pydantic behavior, including variable casting (see example #1 below) and extra field behaviors (see example #2 below). These can be changed through Pydantic's configuration.

The only difference in behavior between LaminDB and Pydantic is strict type checking for `Relationship` fields (see example #3 below), which is implemented in LaminDB.

## Special Cases in Data Validation #1: Variable Casting

LaminDB mirrors Pydantic's default behavior of casting input variables to conform to field types (see details in [Pydantic's documentation](https://docs.pydantic.dev/usage/models/#data-conversion)).

Let's take a look at the default behavior by creating pipeline instances with `int` and `bool` inputs to the `name` field, which is string-typed in the schema.

In [ ]:
# Name (int) is cast to str
pipeline_name_int_to_str = lns.Pipeline(name=1)
type(pipeline_name_int_to_str.name)

In [ ]:
# Name (bool) is cast to str
pipeline_name_bool_to_str = lns.Pipeline(name=True)
type(pipeline_name_int_to_str.name)

## Special Cases in Data Validation #2: Extra Fields

LaminDB also mirror's Pydantic default behavior of accepting extra fields not defined in the schema.

In [ ]:
# No error is raised for the extra field
pipeline = lns.Pipeline(
    name="Test", extra_field="This field is not defined in the schema"
)

## Special Cases in Data Validation #3: Strict Type Checking for Relationship

Differently from Pydantic, LaminDB enforces strict type checking for `Relationship` fields.

Below is a simple example of Pydantic's lenient type checking for `Relationship` fields. Rather than enforcing the `Car` type in the `Wheel.car` field, it only enforces type-checking on the attributes of the input object.

In [ ]:
from sqlmodel import SQLModel, Field, Relationship
from typing import Optional, List


class Car(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

    wheels: List["Wheel"] = Relationship()


class Wheel(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

    car: Optional["Car"] = Relationship()


class Bird(SQLModel, table=False):
    id: str = Field(primary_key=True, default=None)
    name: str

In [ ]:
# Pydantic does not raise a validation error for wrong type in the car field
wheel = Wheel(name="Test Wheel", car=Bird(name="Test"))

LaminDB, on the other hand, enforces strict type checking for `Relationships`.

In [ ]:
with pytest.raises(TypeError) as e:
    exception = e
    run = lns.Run(name="Test Run", pipeline=Bird(name="This is not a Pipeline"))
print(exception.exconly())